In [1]:
from datetime import date
from datetime import datetime
from datetime import timedelta
import os
import subprocess
import urllib
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from matplotlib import style

# Gathering Data for Intermarket Correlation Study

-Data will be scraped from the FRED website and from Yahoo Finance using the yfinance app.

-Data for the 'Pre-COVID' and 'Post-COVID' period will be stored in CSV files. A 'metadata' file will be made to encompass all dates.

# Get Post-COVID Data

In [2]:
today = datetime.today().strftime('%Y-%m-%d')
beginning = '2020-03-19'
if not(os.path.exists('Covid-Study-Data')):
    cmd = 'mkdir ' + os.path.join(os.getcwd(),'Covid-Study-Data')
    print(cmd)
    (status, output) = subprocess.getstatusoutput(cmd)    # try to make a new directory
    print("STATUS:\n", status,"OUTPUT:\n", output)                            # 0 implies error free!
else:
    print("'{}' already exists".format(os.path.join(os.getcwd(), 'Covid-Study-Data')))


'/Users/sambaker/Desktop/School/DA/IS/Covid-Study-Data' already exists


In [3]:
os.chdir('Covid-Study-Data')
tickers = {'EURUSD':'EURUSD=X','JPYUSD':'JPY=X','AUDUSD':'AUDUSD=X','CHFUSD':'CHF=X','CADUSD':'CAD=X','NOKUSD':'NOK=X',
 'SEKUSD':'SEK=X','NZDUSD':'NZDUSD=X','RUBUSD':'RUBUSD=X','CNYUSD':'CNY=X','MXNUSD':'MXN=X','GBPUSD':'GBPUSD=X',
 'BRLUSD':'BRL=X','DXY':'DX-Y.NYB','BTCUSD':'BTC-USD','GOLD':'GLD','S&P500':'^GSPC','DJIA':'^DJI','NASDAQ':'^IXIC'}
for key in tickers.keys():
    tickers[key] = yf.download(tickers[key], start=beginning, end=today, progress=False)[["Adj Close"]]
data = pd.concat(list(tickers.values()), axis=1)
data.columns = tickers.keys()
baseurl = 'https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id='
base2 = '&scale=left&cosd='
base3 = '&coed='
base4 = '&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Daily&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date='
base5 = '&revision_date='
DGS10 = baseurl + 'DGS10' + base2 + beginning + base3 + today + base4 + str(today) + base5 + str(today) + '&nd=1962-01-02'
DFII10 = baseurl + 'DFII10'+ base2 + beginning + base3 + today + base4 + str(today) + base5 + str(today) + '&nd=2003-01-02'
DGS30 = baseurl + 'DGS30'+ base2 + beginning + base3 + today + base4 + str(today) + base5 + str(today) + '&nd=1977-02-15'
DFII30 = baseurl + 'DFII30'+ base2 + beginning + base3 + today + base4 + str(today) + base5 + str(today) + '&nd=2010-02-22'
DGS2 = baseurl + 'DGS2'+ base2 + beginning + base3 + today + base4 + str(today) + base5 + str(today) + '&nd=1976-06-01'
datalinks = {'DGS10':DGS10,'DFII10':DFII10,'DGS30':DGS30,'DFII30':DFII30,'DGS2':DGS2}

In [4]:
for element in datalinks.keys():
    destfile = os.path.join(os.getcwd(), element + '.csv')
    fromfile = datalinks.get(element)
    if not os.path.exists(destfile):
        urllib.request.urlretrieve(fromfile, destfile)
    else:
        print("'{}' already exists".format(destfile))

'/Users/sambaker/Desktop/School/DA/IS/Covid-Study-Data/DGS10.csv' already exists
'/Users/sambaker/Desktop/School/DA/IS/Covid-Study-Data/DFII10.csv' already exists
'/Users/sambaker/Desktop/School/DA/IS/Covid-Study-Data/DGS30.csv' already exists
'/Users/sambaker/Desktop/School/DA/IS/Covid-Study-Data/DFII30.csv' already exists
'/Users/sambaker/Desktop/School/DA/IS/Covid-Study-Data/DGS2.csv' already exists


In [5]:
DGS10 = pd.read_csv('DGS10.csv').set_index('DATE')
DFII10 = pd.read_csv('DFII10.csv').set_index('DATE')
DGS30 = pd.read_csv('DGS30.csv').set_index('DATE')
DFII30 = pd.read_csv('DFII30.csv').set_index('DATE')
DGS2 = pd.read_csv('DGS2.csv').set_index('DATE')
tdata = pd.concat([DGS10, DFII10,DGS30,DFII30,DGS2], axis=1)
tdata = tdata[tdata.DGS10 != '.']
tdata = tdata[tdata.DGS10 != ''].astype(float)
data = data.join(tdata)
data.dropna(inplace=True)
data.to_csv('Postcovid.csv')

# Get Pre-COVID Data

In [6]:
beginning = '2015-02-18'
ending = '2020-02-19'
tickers = {'EURUSD':'EURUSD=X','JPYUSD':'JPY=X','AUDUSD':'AUDUSD=X','CHFUSD':'CHF=X','CADUSD':'CAD=X','NOKUSD':'NOK=X',
 'SEKUSD':'SEK=X','NZDUSD':'NZDUSD=X','RUBUSD':'RUBUSD=X','CNYUSD':'CNY=X','MXNUSD':'MXN=X','GBPUSD':'GBPUSD=X',
 'BRLUSD':'BRL=X','DXY':'DX-Y.NYB','BTCUSD':'BTC-USD','GOLD':'GLD','S&P500':'^GSPC','DJIA':'^DJI','NASDAQ':'^IXIC'}
for key in tickers.keys():
    tickers[key] = yf.download(tickers[key], start=beginning, end=ending, progress=False)[["Adj Close"]]
data = pd.concat(list(tickers.values()), axis=1)
data.columns = tickers.keys()
baseurl = 'https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id='
base2 = '&scale=left&cosd='
base3 = '&coed='
base4 = '&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Daily&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date='
base5 = '&revision_date='
DGS10 = baseurl + 'DGS10' + base2 + beginning + base3 + ending + base4 + str(ending) + base5 + str(ending) + '&nd=1962-01-02'
DFII10 = baseurl + 'DFII10'+ base2 + beginning + base3 + ending + base4 + str(ending) + base5 + str(ending) + '&nd=2003-01-02'
DGS30 = baseurl + 'DGS30'+ base2 + beginning + base3 + ending + base4 + str(ending) + base5 + str(ending) + '&nd=1977-02-15'
DFII30 = baseurl + 'DFII30'+ base2 + beginning + base3 + ending + base4 + str(ending) + base5 + str(ending) + '&nd=2010-02-22'
DGS2 = baseurl + 'DGS2'+ base2 + beginning + base3 + ending + base4 + str(ending) + base5 + str(ending) + '&nd=1976-06-01'
datalinks = {'DGS10':DGS10,'DFII10':DFII10,'DGS30':DGS30,'DFII30':DFII30,'DGS2':DGS2}
for element in datalinks.keys():
    destfile = os.path.join(os.getcwd(), element + 'pre' + '.csv')
    fromfile = datalinks.get(element)
    if not os.path.exists(destfile):
        urllib.request.urlretrieve(fromfile, destfile)
    else:
        print("'{}' already exists".format(destfile))

'/Users/sambaker/Desktop/School/DA/IS/Covid-Study-Data/DGS10pre.csv' already exists
'/Users/sambaker/Desktop/School/DA/IS/Covid-Study-Data/DFII10pre.csv' already exists
'/Users/sambaker/Desktop/School/DA/IS/Covid-Study-Data/DGS30pre.csv' already exists
'/Users/sambaker/Desktop/School/DA/IS/Covid-Study-Data/DFII30pre.csv' already exists
'/Users/sambaker/Desktop/School/DA/IS/Covid-Study-Data/DGS2pre.csv' already exists


In [7]:
DGS10 = pd.read_csv('DGS10pre.csv').set_index('DATE')
DFII10 = pd.read_csv('DFII10pre.csv').set_index('DATE')
DGS30 = pd.read_csv('DGS30pre.csv').set_index('DATE')
DFII30 = pd.read_csv('DFII30pre.csv').set_index('DATE')
DGS2 = pd.read_csv('DGS2pre.csv').set_index('DATE')
tdata = pd.concat([DGS10, DFII10,DGS30,DFII30,DGS2], axis=1)
tdata = tdata[tdata.DGS10 != '.']
tdata = tdata[tdata.DGS10 != ''].astype(float)
data = data.join(tdata)
data.dropna(inplace=True)
data.to_csv('Precovid.csv')

# Metadata

In [8]:
beginning = '2015-02-18'
today = datetime.today().strftime('%Y-%m-%d')
tickers = {'EURUSD':'EURUSD=X','JPYUSD':'JPY=X','AUDUSD':'AUDUSD=X','CHFUSD':'CHF=X','CADUSD':'CAD=X','NOKUSD':'NOK=X',
 'SEKUSD':'SEK=X','NZDUSD':'NZDUSD=X','RUBUSD':'RUBUSD=X','CNYUSD':'CNY=X','MXNUSD':'MXN=X','GBPUSD':'GBPUSD=X',
 'BRLUSD':'BRL=X','DXY':'DX-Y.NYB','BTCUSD':'BTC-USD','GOLD':'GLD','S&P500':'^GSPC','DJIA':'^DJI','NASDAQ':'^IXIC'}
for key in tickers.keys():
    tickers[key] = yf.download(tickers[key], start=beginning, end=today, progress=False)[["Adj Close"]]
data = pd.concat(list(tickers.values()), axis=1)
data.columns = tickers.keys()
baseurl = 'https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id='
base2 = '&scale=left&cosd='
base3 = '&coed='
base4 = '&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Daily&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date='
base5 = '&revision_date='
DGS10 = baseurl + 'DGS10' + base2 + beginning + base3 + today + base4 + str(today) + base5 + str(today) + '&nd=1962-01-02'
DFII10 = baseurl + 'DFII10'+ base2 + beginning + base3 + today + base4 + str(today) + base5 + str(today) + '&nd=2003-01-02'
DGS30 = baseurl + 'DGS30'+ base2 + beginning + base3 + today + base4 + str(today) + base5 + str(today) + '&nd=1977-02-15'
DFII30 = baseurl + 'DFII30'+ base2 + beginning + base3 + today + base4 + str(today) + base5 + str(today) + '&nd=2010-02-22'
DGS2 = baseurl + 'DGS2'+ base2 + beginning + base3 + today + base4 + str(today) + base5 + str(today) + '&nd=1976-06-01'
datalinks = {'DGS10':DGS10,'DFII10':DFII10,'DGS30':DGS30,'DFII30':DFII30,'DGS2':DGS2}
for element in datalinks.keys():
    destfile = os.path.join(os.getcwd(), element + 'meta' + '.csv')
    fromfile = datalinks.get(element)
    urllib.request.urlretrieve(fromfile, destfile)
DGS10 = pd.read_csv('DGS10meta.csv').set_index('DATE')
DFII10 = pd.read_csv('DFII10meta.csv').set_index('DATE')
DGS30 = pd.read_csv('DGS30meta.csv').set_index('DATE')
DFII30 = pd.read_csv('DFII30meta.csv').set_index('DATE')
DGS2 = pd.read_csv('DGS2meta.csv').set_index('DATE')
tdata = pd.concat([DGS10, DFII10,DGS30,DFII30,DGS2], axis=1)
tdata = tdata[tdata.DGS10 != '.']
tdata = tdata[tdata.DGS10 != ''].astype(float)
data = data.join(tdata)
data.dropna(inplace=True)
data.to_csv('data.csv')